# Simple SageMaker AI-Enhanced Fraud Data Generation

This notebook demonstrates SageMaker AI-enhanced fraud data generation without Neptune dependencies.

## Workflow:
1. **Generate 100K base transactions** using enhanced fraud logic
2. **Train SageMaker models** (RandomCutForest + XGBoost) on local data
3. **Generate AI-enhanced transactions** using trained endpoints
4. **Save results locally** for analysis or further use

## Benefits:
- ✅ **Portable**: Runs in any AWS account
- ✅ **Simple**: No VPC or Neptune setup required
- ✅ **Cost-effective**: Only SageMaker training/endpoint costs
- ✅ **Focused**: Pure ML enhancement demonstration

## Step 1: Setup and Configuration

In [ ]:
# AWS Authentication and Setup
import pandas as pd
import boto3
import sagemaker
import os
import sys

# Try different authentication methods in order of preference
print("🔐 Setting up AWS authentication...")

session = None
auth_method = None

# Method 1: Try SageMaker execution role (for SageMaker notebook instances)
try:
    from sagemaker import get_execution_role
    role = get_execution_role()
    session = boto3.Session()
    # Test the session
    session.client('sts').get_caller_identity()
    auth_method = "SageMaker IAM Role"
    print(f"✅ Using SageMaker execution role: {role}")
except:
    pass

# Method 2: Try default AWS credentials (CLI, environment variables, IAM role)
if not session:
    try:
        session = boto3.Session()
        # Test the session
        session.client('sts').get_caller_identity()
        auth_method = "AWS Default Credentials"
        print("✅ Using AWS default credentials (CLI/environment/IAM role)")
    except:
        pass

# Method 3: Manual credentials (if needed)
if not session:
    print("❌ No AWS credentials found")
    print("\n🔧 Setup options:")
    print("   1. Run 'aws configure' in terminal")
    print("   2. Set environment variables: AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY")
    print("   3. Use IAM role if running on EC2/SageMaker")
    print("\n💡 For workshop/temporary credentials, set environment variables:")
    print("")
    print("# Option A: Set in terminal before starting notebook:")
    print("# export AWS_ACCESS_KEY_ID='your_access_key'")
    print("# export AWS_SECRET_ACCESS_KEY='your_secret_key'")
    print("# export AWS_SESSION_TOKEN='your_session_token'  # For temporary credentials")
    print("# export AWS_DEFAULT_REGION='us-west-2'")
    print("")
    print("# Option B: Set in notebook (less secure):")
    print("# os.environ['AWS_ACCESS_KEY_ID'] = 'your_access_key'")
    print("# os.environ['AWS_SECRET_ACCESS_KEY'] = 'your_secret_key'")
    print("# os.environ['AWS_SESSION_TOKEN'] = 'your_session_token'")
    print("# os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'")
    print("# session = boto3.Session()  # Then retry")
    raise Exception("AWS credentials not configured")

# Get AWS account information
try:
    account_id = session.client('sts').get_caller_identity()['Account']
    region = session.region_name or 'us-west-2'  # Default region if not set
    print(f"\n✅ AWS Authentication successful!")
    print(f"   Method: {auth_method}")
    print(f"   Account ID: {account_id}")
    print(f"   Region: {region}")
except Exception as e:
    print(f"❌ AWS authentication failed: {e}")
    raise

In [ ]:
# Import project modules and verify setup
# Add src directory to path
sys.path.append('src')

try:
    from enhanced_fraud_generator import EnhancedFraudGenerator
    from sagemaker_fraud_enhancer import SageMakerFraudEnhancer
    print("✅ Project modules imported successfully")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("\n🔧 Make sure you're running this notebook from the correct directory")
    print("   The 'src' folder should be in the same directory as this notebook")
    raise

print("\n🤖 Simple SageMaker AI-Enhanced Fraud Generator")
print(f"SageMaker Version: {sagemaker.__version__}")
print("\n✅ Ready for AI-enhanced fraud generation!")

## Step 2: Generate Base Dataset (100K Transactions)

In [ ]:
# Initialize the enhanced fraud generator
print("🏭 Initializing Enhanced Fraud Generator...")
generator = EnhancedFraudGenerator('config/enhanced_fraud_rules.yaml')

# Generate complete fraud network
print("\n📊 Generating enhanced fraud network...")
print("This includes:")
print("   • 9 different fraud types")
print("   • Realistic financial institutions")
print("   • Complex fraud patterns")
print("   • Risk scoring")
print("\n⏱️  This may take 2-3 minutes...")

# Generate the network data
network_data = generator.generate_network()
transactions = network_data['transactions'].to_dict('records')

print(f"\n✅ Generated {len(transactions):,} base transactions!")

# Quick analysis
df = pd.DataFrame(transactions)
fraud_count = len(df[df['is_fraud'] == True])
print(f"\n📈 Dataset Summary:")
print(f"   • Total transactions: {len(df):,}")
print(f"   • Fraud transactions: {fraud_count:,}")
print(f"   • Fraud rate: {fraud_count/len(df)*100:.1f}%")
print(f"   • Institutions: {network_data['institutions'].shape[0]}")
print(f"   • Accounts: {network_data['accounts'].shape[0]}")

## Step 3: Save Base Dataset Locally

In [ ]:
# Save the base dataset for SageMaker training
print("💾 Saving base dataset locally...")

# Create output directory
output_dir = 'base_dataset'
os.makedirs(output_dir, exist_ok=True)

# Save transactions
transactions_df = pd.DataFrame(transactions)
transactions_df.to_csv(f'{output_dir}/transactions.csv', index=False)

# Save institutions and accounts from network data
institutions_df = network_data['institutions']
institutions = institutions_df.to_dict('records')
institutions_df.to_csv(f'{output_dir}/institutions.csv', index=False)

accounts_df = network_data['accounts']
accounts = accounts_df.to_dict('records')
accounts_df.to_csv(f'{output_dir}/accounts.csv', index=False)

print(f"\n✅ Base dataset saved to {output_dir}/:")
print(f"   • transactions.csv ({len(transactions):,} records)")
print(f"   • institutions.csv ({len(institutions)} records)")
print(f"   • accounts.csv ({len(accounts)} records)")
print("\n📊 Ready for SageMaker training!")

## Step 4: Train SageMaker Models

In [ ]:
# Initialize SageMaker AI enhancer
print("🤖 Initializing SageMaker AI Enhancer...")
sagemaker_enhancer = SageMakerFraudEnhancer()

# Load the base dataset
print("\n📂 Loading base dataset for SageMaker training...")
sagemaker_enhancer.load_existing_data('base_dataset')

print("\n✅ Dataset loaded successfully!")

In [ ]:
# Train SageMaker models
print("🚀 Training SageMaker models...")
print("\nThis will:")
print("   • Create S3 bucket for SageMaker")
print("   • Upload training data to S3")
print("   • Train Random Cut Forest for anomaly detection")
print("   • Train XGBoost for fraud classification")
print("   • Deploy both models as endpoints")
print("\n⏱️  Expected time: 10-15 minutes")
print("💰 Cost: ~$2-3 for training + ~$0.10/hour for endpoints")

# Execute training
sagemaker_enhancer.train_sagemaker_models()

print("\n✅ SageMaker models trained and deployed successfully!")
print("\n🎯 Ready for AI-enhanced transaction generation!")

## Step 5: Generate AI-Enhanced Transactions

In [ ]:
# Generate AI-enhanced transactions
print("🎯 Generating AI-enhanced transactions...")
print("\n📊 SageMaker enhancements include:")
print("   • Random Cut Forest anomaly detection")
print("   • XGBoost fraud classification")
print("   • Real-time endpoint inference")
print("   • Enterprise-grade ML models")
print("\n⏱️  This may take 5-8 minutes for 30K transactions...")

# Generate enhanced transactions
enhanced_count = 30000
ai_transactions = sagemaker_enhancer.generate_ai_enhanced_transactions(enhanced_count)

print(f"\n✅ Generated {len(ai_transactions):,} AI-enhanced transactions!")

## Step 6: Analyze AI-Enhanced Results

In [ ]:
# Analyze the AI-enhanced dataset
ai_df = pd.DataFrame(ai_transactions)

print(f"📈 AI-Enhanced Dataset Analysis:")
print(f"  Total AI Transactions: {len(ai_df):,}")

# Fraud statistics
ai_fraud_df = ai_df[ai_df['is_fraud'] == True]
print(f"\n🚨 AI Fraud Statistics:")
print(f"  AI Fraud Transactions: {len(ai_fraud_df):,}")
print(f"  AI Fraud Rate: {len(ai_fraud_df)/len(ai_df)*100:.2f}%")

# AI-specific metrics
print(f"\n🤖 SageMaker Enhancement Metrics:")
print(f"  Average AI Confidence: {ai_df['ai_confidence'].mean():.3f}")
print(f"  Average Pattern Similarity: {ai_df['pattern_similarity'].mean():.3f}")
print(f"  Generation Method: {ai_df['generation_method'].iloc[0]}")

# Compare with base dataset
base_fraud_rate = fraud_count / len(df) * 100
ai_fraud_rate = len(ai_fraud_df) / len(ai_df) * 100

print(f"\n📊 Comparison:")
print(f"  Base Dataset Fraud Rate: {base_fraud_rate:.2f}%")
print(f"  AI-Enhanced Fraud Rate: {ai_fraud_rate:.2f}%")
print(f"  Enhancement Factor: {ai_fraud_rate/base_fraud_rate:.2f}x")

# Show sample transactions
print("\n💸 Sample AI-Enhanced Transactions:")
display(ai_df[['transaction_id', 'amount', 'fraud_type', 'ai_confidence', 'generation_method']].head())

## Step 7: Save AI-Enhanced Dataset

In [ ]:
# Save AI-enhanced dataset
print("💾 Saving AI-enhanced dataset...")

# Save enhanced data
sagemaker_enhancer.save_sagemaker_enhanced_data(ai_transactions, 'ai_enhanced_output')

print("\n✅ AI-enhanced dataset saved successfully!")
print("\n📁 Output files:")
print("   • ai_enhanced_output/transactions.csv - AI-enhanced transactions")
print("   • ai_enhanced_output/institutions.csv - Institution data")
print("   • ai_enhanced_output/accounts.csv - Account data")
print("   • ai_enhanced_output/sagemaker_summary.json - Enhancement summary")

# Show file sizes
import os
for file in ['transactions.csv', 'institutions.csv', 'accounts.csv', 'sagemaker_summary.json']:
    path = f'ai_enhanced_output/{file}'
    if os.path.exists(path):
        size = os.path.getsize(path)
        print(f"   📄 {file}: {size:,} bytes")

## Step 8: Cleanup (Optional)

In [ ]:
# IMPORTANT: Cleanup SageMaker endpoints to avoid ongoing charges
print("💰 SageMaker Endpoint Cleanup")
print("=" * 40)
print("\n⚠️  SageMaker endpoints cost ~$0.10/hour each when running")
print("   Cleanup is recommended when done with the workshop")

print("\n🧹 To cleanup endpoints, uncomment and run:")
print("# sagemaker_enhancer.cleanup_endpoints()")

print("\n📊 Or check current endpoints:")
sagemaker_client = boto3.client('sagemaker')
try:
    endpoints = sagemaker_client.list_endpoints()['Endpoints']
    if endpoints:
        print("\n📋 Current SageMaker endpoints:")
        for ep in endpoints:
            print(f"   • {ep['EndpointName']} - {ep['EndpointStatus']}")
    else:
        print("\n✅ No active endpoints found")
except Exception as e:
    print(f"\n❌ Could not list endpoints: {e}")

print("\n💡 Remember: Endpoints can be recreated from saved models if needed later")

## Summary

✅ **Simple SageMaker AI Enhancement Completed:**
1. Generated enhanced fraud network with institutions, accounts, and transactions
2. Trained RandomCutForest for anomaly detection
3. Trained XGBoost for fraud classification
4. Generated 30,000 AI-enhanced transactions
5. Saved all results locally for analysis

🤖 **SageMaker Benefits Demonstrated:**
- **Enterprise ML**: Production-ready anomaly detection and classification
- **Scalability**: Can handle millions of transactions
- **Real-time**: Live endpoint inference for fraud scoring
- **Portability**: Runs in any AWS account without VPC setup

💰 **Cost Summary:**
- Training: ~$2-3 per session
- Endpoints: ~$0.10/hour each (remember to cleanup!)
- Storage: Minimal S3 costs

📊 **Next Steps:**
- Analyze AI-enhanced transaction patterns
- Compare fraud detection accuracy
- Scale to larger datasets
- Integrate with production systems
- Add Neptune for graph analysis (optional)

🎯 **Integration Ready:**
This simplified approach can easily be extended with:
- Neptune graph database (add VPC when needed)
- QuickSight dashboards
- Real-time streaming
- Production ML pipelines